In [1]:
import $exec.^.scripts.burgers_eq

import $exec.$                   

In [2]:
val res = burgers_eq.solve_1d(
    num_data = 100, 
    num_neurons = Seq(40, 30, 20),
    iterations = 50000, 
    pde_wt = 20.0, 
    reg = 0.0001,
    viscosity_param = 0.008, 
    optimizer = tf.train.AdaDelta(0.1f), 
    quadrature_scheme = "GaussLegendre", 
    tempdir = root/'media/'disk2/'scratch/'mandar/'tmp
)

SLF4J: Class path contains multiple SLF4J providers.
SLF4J: Found provider [org.slf4j.log4j12.Log4j12ServiceProvider@673392ef]
SLF4J: Found provider [org.slf4j.helpers.NOPServiceProvider@7e5db16d]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual provider is of type [org.slf4j.log4j12.Log4j12ServiceProvider@673392ef]


Test Data Shapes: [441, 2]


res: (dynamics.PDESystem[Float, Float, Float], dynamics.PDESystem.Model[Float, Float, Float], data.SupervisedDataSet[Tensor[Float], Tensor[Float]], DelauneySurface, Tensor[Float], Tensor[Float]) = (
  io.github.tailhq.dynaml.tensorflow.dynamics.PDESystem@4aa68973,
  Model(
    io.github.tailhq.dynaml.tensorflow.models.TFModel@57b90c3d,
    "Output",
    Vector()
  ),
  SupervisedDataSet(
    io.github.tailhq.dynaml.tensorflow.data.DataSet@3717d846,
    io.github.tailhq.dynaml.tensorflow.data.DataSet@2262d5f6
  ),
  io.github.tailhq.dynaml.graphics.plot3d.DelauneySurface@70d4d8ad,
  Tensor[Float, [441, 2]],
  Tensor[Float, [441, 1]]
)

In [3]:
plot3d.show(res._4)


------------------------------------
Rotate     : Left click and drag mouse
Scale      : Roll mouse wheel
Z Shift    : Right click and drag mouse
Animate    : Double left click
Screenshot : Press 's'

------------------------------------
